In [44]:
import matplotlib.pyplot as plt
from os.path import join,isdir
import numpy as np
from nilearn import plotting, input_data, image
import nibabel as nib

def concat_images(imga, imgb):
    """
    Combines two color image ndarrays side-by-side.
    """
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    max_height = np.max([ha, hb])
    total_width = wa+wb
    new_img = np.zeros(shape=(max_height, total_width, 3), dtype=np.uint8)
    new_img[:ha,:wa]=imga
    new_img[:hb,wa:wa+wb]=imgb
    return new_img

def concat_n_images(image_path_list):
    """
    Combines N color images from a list of image paths.
    """
    output = None
    for i, img_path in enumerate(image_path_list):
        img = plt.imread(img_path)[:,:,:3]
        if i==0:
            output = img
        else:
            output = concat_images(output, img)
    return output


def plotrsn10(nii_ff,workdir,resultjpg_ff, threshold, stat_or_mask):
    '''
    plotrsn10('PNAS_Smith09_rsn10.nii.gz',r'c:\temp',r'c:\temp\test.jpg')
    '''
    z=[8,-4,-10,30,-34,50,14,22,46,48]
    ii = 0
    images=[]
    fig=plt.figure(figsize=(4,6),dpi=300)
    ndim = nib.load(nii_ff).get_data().ndim

    for ii in range(10):
        if ndim == 3:
            img = nii_ff.replace('rsn0','rsn%d' % ii)
        else:
            img = image.index_img(nii_ff, ii)
            # img is now an in-memory 3D img
        tempimage= join(workdir,'RSN%02d.jpg' % (ii+1))
        
        if stat_or_mask == 0:
            display = plotting.plot_stat_map(img, figure=fig, threshold=threshold, display_mode="z", 
                                             cut_coords=[(z[ii])],colorbar=False, annotate=False, black_bg=True)
        else:
            display = plotting.plot_roi(img, figure=fig, display_mode="z", cut_coords=[(z[ii])],
                       colorbar=False, annotate=False, black_bg=True)
        #display.annotate(size=30)
        display.savefig(tempimage)
        images.append(tempimage)
        plt.clf()
        ii += 1
    plt.close()
    #row1 = concat_n_images(images[0:5])
    #row2 = concat_n_images(images[5:10])

    #output = np.vstack((row1,255*np.ones((10,row1.shape[1],3),dtype=np.uint8),row2))
    output = concat_n_images(images)
    fig=plt.figure(figsize=(output.shape[0]//30,output.shape[1]//30),dpi=100)
    plt.axis('off')
    plt.imshow(output)
    fig.savefig(resultjpg_ff, bbox_inches='tight')

In [20]:
plotrsn10(r'c:\temp\rsn10_drzstat.nii.gz',r'c:\temp',r'c:\temp\test.jpg',4)

In [45]:
plotrsn10(r'D:\ABIDE_2017\S0217_114505_rsn10_drzstat_N100_tfce\rsn0\twosample_tfce_corrp_tstat1.nii.gz',r'c:\temp',r'c:\temp\test.jpg',0.9)

C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
C:\Anaconda3\lib\site-packages\numpy\ma\core.py:4185: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")
